# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 22 2022 11:30AM,253678,10,0086265574,ISDIN Corporation,Released
1,Dec 22 2022 11:30AM,253677,10,0086265534,ISDIN Corporation,Released
2,Dec 22 2022 11:24AM,253675,10,0086264778,ISDIN Corporation,Released
3,Dec 22 2022 11:24AM,253675,10,0086265487,ISDIN Corporation,Released
4,Dec 22 2022 11:24AM,253675,10,0086265488,ISDIN Corporation,Released
5,Dec 22 2022 11:24AM,253675,10,0086265515,ISDIN Corporation,Released
6,Dec 22 2022 11:24AM,253675,10,0086265519,ISDIN Corporation,Released
7,Dec 22 2022 11:24AM,253675,10,0086265526,ISDIN Corporation,Released
8,Dec 22 2022 11:24AM,253675,10,0086265529,ISDIN Corporation,Released
9,Dec 22 2022 11:24AM,253675,10,0086265527,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,253673,Released,7
25,253675,Released,30
26,253676,Released,6
27,253677,Released,1
28,253678,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
253673,NaN,7.0
253675,NaN,30.0
253676,NaN,6.0
253677,NaN,1.0
253678,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
253622,0.0,1.0
253624,0.0,1.0
253625,0.0,1.0
253628,0.0,1.0
253629,1.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
253622,0,1
253624,0,1
253625,0,1
253628,0,1
253629,1,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,253622,0,1
1,253624,0,1
2,253625,0,1
3,253628,0,1
4,253629,1,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,253622,,1
1,253624,,1
2,253625,,1
3,253628,,1
4,253629,1,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 22 2022 11:30AM,253678,10,ISDIN Corporation
1,Dec 22 2022 11:30AM,253677,10,ISDIN Corporation
2,Dec 22 2022 11:24AM,253675,10,ISDIN Corporation
32,Dec 22 2022 11:24AM,253673,10,ISDIN Corporation
39,Dec 22 2022 11:23AM,253676,10,"Methapharm, Inc."
45,Dec 22 2022 11:08AM,253672,10,Bio-PRF
52,Dec 22 2022 11:07AM,253671,10,Eywa Pharma Inc.
66,Dec 22 2022 10:53AM,253669,10,"Citieffe, Inc."
69,Dec 22 2022 10:47AM,253668,21,"NBTY Global, Inc."
70,Dec 22 2022 10:33AM,253667,15,"Mizner Bioscience, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Dec 22 2022 11:30AM,253678,10,ISDIN Corporation,,1
1,Dec 22 2022 11:30AM,253677,10,ISDIN Corporation,,1
2,Dec 22 2022 11:24AM,253675,10,ISDIN Corporation,,30
3,Dec 22 2022 11:24AM,253673,10,ISDIN Corporation,,7
4,Dec 22 2022 11:23AM,253676,10,"Methapharm, Inc.",,6
5,Dec 22 2022 11:08AM,253672,10,Bio-PRF,,7
6,Dec 22 2022 11:07AM,253671,10,Eywa Pharma Inc.,,14
7,Dec 22 2022 10:53AM,253669,10,"Citieffe, Inc.",,3
8,Dec 22 2022 10:47AM,253668,21,"NBTY Global, Inc.",,1
9,Dec 22 2022 10:33AM,253667,15,"Mizner Bioscience, LLC",1,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 22 2022 11:30AM,253678,10,ISDIN Corporation,1,
1,Dec 22 2022 11:30AM,253677,10,ISDIN Corporation,1,
2,Dec 22 2022 11:24AM,253675,10,ISDIN Corporation,30,
3,Dec 22 2022 11:24AM,253673,10,ISDIN Corporation,7,
4,Dec 22 2022 11:23AM,253676,10,"Methapharm, Inc.",6,
5,Dec 22 2022 11:08AM,253672,10,Bio-PRF,7,
6,Dec 22 2022 11:07AM,253671,10,Eywa Pharma Inc.,14,
7,Dec 22 2022 10:53AM,253669,10,"Citieffe, Inc.",3,
8,Dec 22 2022 10:47AM,253668,21,"NBTY Global, Inc.",1,
9,Dec 22 2022 10:33AM,253667,15,"Mizner Bioscience, LLC",3,1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 22 2022 11:30AM,253678,10,ISDIN Corporation,1,
1,Dec 22 2022 11:30AM,253677,10,ISDIN Corporation,1,
2,Dec 22 2022 11:24AM,253675,10,ISDIN Corporation,30,
3,Dec 22 2022 11:24AM,253673,10,ISDIN Corporation,7,
4,Dec 22 2022 11:23AM,253676,10,"Methapharm, Inc.",6,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Dec 22 2022 11:30AM,253678,10,ISDIN Corporation,1,NaN
1,Dec 22 2022 11:30AM,253677,10,ISDIN Corporation,1,NaN
2,Dec 22 2022 11:24AM,253675,10,ISDIN Corporation,30,NaN
3,Dec 22 2022 11:24AM,253673,10,ISDIN Corporation,7,NaN
4,Dec 22 2022 11:23AM,253676,10,"Methapharm, Inc.",6,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 22 2022 11:30AM,253678,10,ISDIN Corporation,1,0.0
1,Dec 22 2022 11:30AM,253677,10,ISDIN Corporation,1,0.0
2,Dec 22 2022 11:24AM,253675,10,ISDIN Corporation,30,0.0
3,Dec 22 2022 11:24AM,253673,10,ISDIN Corporation,7,0.0
4,Dec 22 2022 11:23AM,253676,10,"Methapharm, Inc.",6,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,4819485,146,1.0
15,507333,5,11.0
17,760877,3,0.0
19,507284,36,0.0
21,253668,1,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,4819485,146,1.0
1,15,507333,5,11.0
2,17,760877,3,0.0
3,19,507284,36,0.0
4,21,253668,1,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,146,1.0
1,15,5,11.0
2,17,3,0.0
3,19,36,0.0
4,21,1,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,146.0
1,15,Released,5.0
2,17,Released,3.0
3,19,Released,36.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,17,19,21
Status,,,,,
Executing,1.0,11.0,0.0,0.0,0.0
Released,146.0,5.0,3.0,36.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,17,19,21
0,Executing,1.0,11.0,0.0,0.0,0.0
1,Released,146.0,5.0,3.0,36.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,17,19,21
0,Executing,1.0,11.0,0.0,0.0,0.0
1,Released,146.0,5.0,3.0,36.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()